In [1]:
from sifce import datatools, sims, network, tv
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical
from astropy.cosmology import Planck18
pd.set_option('display.max_columns', None)

In [2]:
net = network.Network(ifo_uptime_file="segment_list.segs",ifos=["H1", "L1", "V1"], psd_config_file="net_psds.psds")
net.load_psds()
net.psds_for_time(5)

{'H1': <pycbc.types.frequencyseries.FrequencySeries at 0x7fb3e1d471f0>,
 'L1': <pycbc.types.frequencyseries.FrequencySeries at 0x7fb05de35210>,
 'V1': <pycbc.types.frequencyseries.FrequencySeries at 0x7fb05de35ff0>}

In [3]:
prior_gw150914 = dict(mass_1= DeltaFunction(35),
                     mass_2= DeltaFunction(35),
                     a_1 = DeltaFunction(0.5),
                     a_2 = DeltaFunction(0.5),
                     tilt_1 = DeltaFunction(1.4),
                     tilt_2 = DeltaFunction(1.4),
                     phi_12 = DeltaFunction(3.09),
                     phi_jl = DeltaFunction(5.21),
                     theta_jn =  Sine(name='theta_jn'),
                     phase = DeltaFunction(1.04),
                     reference_frequency = DeltaFunction(20),
                     dec=Cosine(name='dec'),
                     ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                     psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
                     geocent_time = DeltaFunction(5),
                     luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'))

In [4]:
prior_obj = PriorDict(dictionary = prior_gw150914)
pop = sims.SimulationSet(distribution=prior_obj, cosmology_model=Planck18, network=net)
pop.sample_distribution(100)

In [5]:
import copy
list_rows_sky = list()
for index, row in pop.simulations_dataframe.iterrows():
    row = copy.copy(row)
    row["luminosity_distance"]=1
    list_rows_sky.append(pop.calc_snrs_sky(row, N=10, benchmark=True))

In [6]:
pop.simulations_dataframe = pd.concat(list_rows_sky, ignore_index=True)

In [7]:
np.median(pop.simulations_dataframe["opt_snr_net"])

1.077411571258428

In [8]:
pop.simulations_dataframe

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,intrinsic_index,reference_snr_index,cal_idx,opt_snr_H1,mf_snr_H1,opt_snr_L1,mf_snr_L1,opt_snr_V1,mf_snr_V1,opt_snr_net,mf_snr_net,iota,phi_1,phi_2
0,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,1.791529,-0.223984,1.645512,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.657830,1.315660e+00,0.504511,4.401188e-16,0.480116,1.010261,0.958010,1.658791,2.263278,5.768989,2.575803
1,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,4.995804,-0.789059,1.599095,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.631465,7.032682e-16,0.668584,1.337168e+00,0.699984,1.338577,1.155738,1.892037,2.263278,5.768989,2.575803
2,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,4.976595,-1.127706,1.657228,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.497079,1.228422e-15,0.602104,1.204209e+00,0.702920,1.218668,1.050577,1.713263,2.263278,5.768989,2.575803
3,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,2.181509,-0.860007,0.383279,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.945101,1.409656e-15,0.778889,1.557778e+00,0.536841,1.666910,1.337192,2.281504,2.263278,5.768989,2.575803
4,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,0.569567,-0.226379,1.353544,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.462352,1.140593e-15,0.260594,5.211884e-01,0.442361,0.595876,0.690914,0.791648,2.263278,5.768989,2.575803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,1.200068,0.825551,2.369514,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.584717,1.898735e-15,0.778540,1.557080e+00,0.858894,1.564597,1.298352,2.207366,0.779202,5.763651,2.570466
996,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,2.655990,-0.953001,1.501589,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.981925,1.963849e+00,0.881936,2.265925e-15,0.466881,2.132855,1.399988,2.899271,0.779202,5.763651,2.570466
997,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,0.819379,-0.044217,0.011736,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.455473,9.109468e-01,0.505172,3.516343e-15,0.408011,1.033481,0.793175,1.377645,0.779202,5.763651,2.570466
998,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,5.316738,-0.947060,2.698508,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.565598,1.131196e+00,0.504647,4.399997e-16,0.201953,1.462983,0.784446,1.849304,0.779202,5.763651,2.570466


In [9]:
pop.simulations_dataframe[pop.simulations_dataframe["opt_snr_net"] >= 1]

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,intrinsic_index,reference_snr_index,cal_idx,opt_snr_H1,mf_snr_H1,opt_snr_L1,mf_snr_L1,opt_snr_V1,mf_snr_V1,opt_snr_net,mf_snr_net,iota,phi_1,phi_2
1,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,4.995804,-0.789059,1.599095,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.631465,7.032682e-16,0.668584,1.337168e+00,0.699984,1.338577,1.155738,1.892037,2.263278,5.768989,2.575803
2,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,4.976595,-1.127706,1.657228,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.497079,1.228422e-15,0.602104,1.204209e+00,0.702920,1.218668,1.050577,1.713263,2.263278,5.768989,2.575803
3,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,2.181509,-0.860007,0.383279,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.945101,1.409656e-15,0.778889,1.557778e+00,0.536841,1.666910,1.337192,2.281504,2.263278,5.768989,2.575803
5,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,0.145672,-0.603769,2.088841,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.646402,4.293857e-16,0.711131,1.422261e+00,0.400431,1.663338,1.041099,2.188497,2.263278,5.768989,2.575803
9,35.0,35.0,0.42901,-0.24234,0.084984,-0.415941,0.264141,0.084984,2.257947,1.04,2.411742,0.209232,1.677903,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.721947,1.443894e+00,0.652120,1.191738e-15,0.780980,1.325502,1.247556,1.960047,2.263278,5.768989,2.575803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,1.460380,-0.675883,2.485708,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.749408,1.498815e+00,0.450537,5.544501e-16,0.658517,0.966760,1.094642,1.783556,0.779202,5.763651,2.570466
994,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,4.775062,-0.242547,0.184846,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.618131,1.236261e+00,0.684075,1.298364e-15,0.721483,1.370089,1.170718,1.845396,0.779202,5.763651,2.570466
995,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,1.200068,0.825551,2.369514,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.584717,1.898735e-15,0.778540,1.557080e+00,0.858894,1.564597,1.298352,2.207366,0.779202,5.763651,2.570466
996,35.0,35.0,0.42771,-0.244626,0.084984,-0.414526,0.266357,0.084984,0.773863,1.04,2.655990,-0.953001,1.501589,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,1.4,1.4,3.09,5.21,20.0,NaN,NaN,NaN,0.981925,1.963849e+00,0.881936,2.265925e-15,0.466881,2.132855,1.399988,2.899271,0.779202,5.763651,2.570466


In [10]:
pop.simulations_dataframe[pop.simulations_dataframe["opt_snr_net"] >= 2]

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,intrinsic_index,reference_snr_index,cal_idx,opt_snr_H1,mf_snr_H1,opt_snr_L1,mf_snr_L1,opt_snr_V1,mf_snr_V1,opt_snr_net,mf_snr_net,iota,phi_1,phi_2


In [11]:
list_rows_distance = list()
for index, row in pop.simulations_dataframe.iterrows():
    list_rows_distance.append(pop.sample_distances(row, N=10))

In [12]:
pop.simulations_dataframe = pd.concat(list_rows_distance, ignore_index=True)

In [13]:
pop.simulations_dataframe[pop.simulations_dataframe["opt_snr_net"] >= 2]

,mass_1,mass_2,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,theta_jn,phase,ra,dec,psi,geocent_time,luminosity_distance,redshift,chirp_mass,total_mass,mass_ratio,symmetric_mass_ratio,mass_1_source,mass_2_source,chirp_mass_source,total_mass_source,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,reference_frequency,intrinsic_index,reference_snr_index,cal_idx,opt_snr_H1,mf_snr_H1,opt_snr_L1,mf_snr_L1,opt_snr_V1,mf_snr_V1,opt_snr_net,mf_snr_net,iota,phi_1,phi_2


In [14]:
def step(row):
    if row["opt_snr_net"] >= 11:
        return 1
    return 0

In [15]:
# Timevol = tv.TimeVolume(cosmology=Planck18, simulation=pop, function_detection=step)

In [16]:
# Timevol.sensitive_volume_calc()